# Authenticating our Python script

In [1]:
import twitter
twitter_api = twitter.Api(consumer_key='YCItuzSp8kdKBq81Tj4mmGBZm',
                  consumer_secret='fb1ZnmFVKTG54wpAamnxNt7GRaFa9vYBIvUAuCoAfaQWTw5Epd',
                  access_token_key='1279054229819154433-1thc23GnRx4Hrh0DR6xTKhBhVXIQAv',
                  access_token_secret='GI8oRzX0Ir68JAVUT4IqK7dKtx6uiedo0NxQlN8RWxjyL')


# test authentication
verify = twitter_api.VerifyCredentials()
print(verify)

{"created_at": "Fri Jul 03 14:08:13 +0000 2020", "default_profile": true, "default_profile_image": true, "followers_count": 1, "id": 1279054229819154433, "id_str": "1279054229819154433", "name": "Darsh Chandak", "profile_background_color": "F5F8FA", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_image_url_https": "https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "ChandakDarsh"}


# Creating the function to build the Test set

In [2]:

def buildTestSet(search_keyword):
    try:
        tweets_fetched = twitter_api.GetSearch(search_keyword, count = 100)
        
        print("Fetched " + str(len(tweets_fetched)) + " tweets for the term " + search_keyword)
        
        return [{"text":status.text, "label":None} for status in tweets_fetched]
    except:
        print("Unfortunately, something went wrong..")
        return None

# Testing Function

In [3]:
search_term = "Covid-19"
testDataSet = buildTestSet(search_term)

print(testDataSet[0:4])
# print(tweet["text"]+"  "+tweet["label"])
# print(testDataSet)

Fetched 100 tweets for the term Covid-19
[{'text': "Schools can't even control lice, you think they can control Covid-19.", 'label': None}, {'text': 'Your reminder that schools in Germany, Norway, and Denmark can open safely because their leaders contained Covid-19… https://t.co/YrQ9jrWW0q', 'label': None}, {'text': 'The media cannot admit that we have effective COVID-19 treatments because then they wouldn’t be able to continue th… https://t.co/MO92AeLbN7', 'label': None}, {'text': 'Pain is permanently haunting me. I lost a dear friend of mine, whose name shall not be revealed for their privacy,… https://t.co/2hSZ5y90VC', 'label': None}]


# Preparing The Training Set

In [4]:
def buildTrainingSet(corpusFile, tweetDataFile):
    import csv
    import time

    corpus = []

    with open(corpusFile,'rt') as csvfile:
        lineReader = csv.reader(csvfile,delimiter=',', quotechar="\"")
        for row in lineReader:
            corpus.append({"tweet_id":row[2], "label":row[1], "topic":row[0]})

    rate_limit = 180
    sleep_time = 900/180

    trainingDataSet = []

    i=0
    for tweet in corpus:
        try:
            i=i+1
            status = twitter_api.GetStatus(tweet["tweet_id"])
            # print("Tweet fetched: ["+i+"] : "+ status.text)
            print("Tweet fetched: ["+str(i)+"] : "+ status.text)
            tweet["text"] = status.text
            trainingDataSet.append(tweet)
            time.sleep(sleep_time)
            with open(tweetDataFile,'w',newline='') as csvfile:
                linewriter = csv.writer(csvfile,delimiter=',',quotechar="\"")
                for tweet in trainingDataSet:
                    try:
                        linewriter.writerow([tweet["tweet_id"], tweet["text"], tweet["label"], tweet["topic"]])
                    except Exception as e:
                        print(e)
        except:
            i=i-1
            # print("Tweet not fetched: ["+str(i)+"]")
            continue
    # now we write them to the empty CSV file
    
    return trainingDataSet

In [45]:

corpusFile = "Datasets/corpus_test.csv"
tweetDataFile = "Datasets/tweetDataFile_test.csv"
print("corpus File:"+corpusFile)
trainingData = buildTrainingSet(corpusFile, tweetDataFile)
print("\n\n\tDone")